In [1]:
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import json
import pprint
import time

In [2]:
consumer_key = 'RipArnaZgfpTQRhtIQhMtEsTy'
consumer_secret = 'ZD1uNeKgT7gR3KG9hVLcjlcjvJIFSVK9z4nXcTJ4sa3XvwVlMY'
access_token = '570510531-pXvq1mzoUGkM2xQtTnJE3GSBtRRV4VlqrTww7QBp'
access_secret = '37nPOzulSRtEeCrqNzbGsbN1VDRE3rzIuIbWcgz40Gfrz'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

### Scrape 

In [73]:
def prepare(name, dic):
    return pd.DataFrame(dic.values(), index = dic.keys(), columns=[name]).reset_index()

def search():
    from_ = []
    to_ = []
    type_ = []
    followers = {}
    listed = {}
    retweets_count = {}
    for tweet in tweepy.Cursor(api.search, q="#bitcoin").items(1000):
        #user who tweets
        user = tweet._json.get("user").get("screen_name")

        #record his/her follower_count, listed_count
        followers[user] = tweet._json.get("user").get("followers_count")
        listed[user] = tweet._json.get("user").get("listed_count")

        #record the tweets' retweet count, used ADDing
        if user in retweets_count:
            retweets_count[user] = retweets_count[user] + int(tweet._json.get("retweet_count"))
        else:
            retweets_count[user] = int(tweet._json.get("retweet_count"))

        #reply to sb
        reply_to = tweet._json.get("in_reply_to_screen_name")
        if reply_to:
            from_.append(user)
            to_.append(reply_to)
            type_.append('reply')

        #mentions sb
        mention_list = tweet._json.get("entities").get("user_mentions")
        if len(mention_list)!=0:
            for mention in mention_list:
                from_.append(user)
                to_.append(mention.get("screen_name"))
                type_.append('mention')

        # RT sb
        retweeted = tweet._json.get("retweeted_status")
        if retweeted:
            from_.append(user)
            to_.append(retweeted.get("user").get("screen_name"))
            type_.append('RT')

            # add the retweeted user's followers_count, listed_count
            followers[retweeted.get("user").get("screen_name")] = retweeted.get("user").get("followers_count")
            listed[retweeted.get("user").get("screen_name")] = retweeted.get("user").get("listed_count")

            # add the original tweet
            from_.append(retweeted.get("user").get("screen_name"))
            to_.append(retweeted.get("user").get("screen_name"))
            type_.append('Tweet')

        # No reply, mention or retweet
        if (reply_to is None) | (len(mention_list)==0) | (retweeted is None) :
            from_.append(user)
            to_.append(user)
            type_.append('Tweet')
    result = pd.DataFrame({'Column1':from_, 'Column2':to_, 'Column3':type_})
    result = result.drop_duplicates()
    return result, prepare('follwers', followers), prepare('listed_count', listed), prepare('retweets', retweets_count)

In [ ]:
tweets = []
f = []
l = []
r = []

In [78]:
# The twitter API has limit on queries in a 15-min window. I put the program to sleep when scraping every 1000 tweets
for i in range(5):
    result, follower_count, listed_count, retweets_count = search()
    tweets.append(result)
    f.append(follower_count)
    l.append(listed_count)
    r.append(retweets_count)
    print('sleeping')
    time.sleep(60*10)
    print('wake up')

sleeping
wake up
sleeping
wake up
sleeping
wake up
sleeping
wake up
sleeping
wake up


In [93]:
t = tweets[0]
f_ = f[0]
l_ = l[0]
r_ = r[0]
for i in range(1,6):
    t = t.append(tweets[i])
    f_ = f_.append(f[i])
    l_ = l_.append(l[i])
    r_ = r_.append(r[i])

In [90]:
t.drop_duplicates().to_csv("tweets_new.csv")
f_.to_csv('followers.csv')
l_.to_csv('linked.csv')
r_.to_csv('retweets.csv')